In [20]:
import sys
sys.path.append("../")

import subprocess
import datetime
from joblib import Parallel, delayed

import json
import os

import matplotlib.pyplot as plt
from tqdm import tqdm
import optuna

from run import *

In [21]:
import contextlib
from typing import Optional
import joblib
from tqdm.auto import tqdm

@contextlib.contextmanager
def tqdm_joblib(total: Optional[int] = None, **kwargs):

    pbar = tqdm(total=total, miniters=1, smoothing=0, **kwargs)

    class TqdmBatchCompletionCallback(joblib.parallel.BatchCompletionCallBack):
        def __call__(self, *args, **kwargs):
            pbar.update(n=self.batch_size)
            return super().__call__(*args, **kwargs)

    old_batch_callback = joblib.parallel.BatchCompletionCallBack
    joblib.parallel.BatchCompletionCallBack = TqdmBatchCompletionCallback

    try:
        yield pbar
    finally:
        joblib.parallel.BatchCompletionCallBack = old_batch_callback
        pbar.close()

In [29]:
case_num = 1000

cases = [
    (f"../tools/in/{seed:04}.txt", f"../tools/out/{seed:04}.txt")
    for seed in range(case_num)
]

def objective(trial: optuna.trial.Trial) -> float:
    overflow_alpha = 2.016646721749814
    overflow_alpha_all = 2.7694849713061416
    overflow_alpha_all_refill = 0.5531191229327318
    overflow_alpha_refill = 0.25378716369770815
    # overflow_alpha_refill = trial.suggest_float("overflow_alpha_refill", low=0.1, high=0.9)
    # overflow_alpha = trial.suggest_float("overflow_alpha", low=1., high=3.)
    # overflow_alpha_all_refill = trial.suggest_float("overflow_alpha_all_refill", low=0.1, high=0.9)
    # overflow_alpha_all = trial.suggest_float("overflow_alpha_all", low=1., high=3.)

    cancel_alpha = 3.9025157664873658
    cancel_alpha_all = 5.034673414949709
    cancel_alpha_all_refill = 1.070299470579473
    cancel_alpha_refill = 1.000910439816072
    # cancel_alpha_refill = trial.suggest_float("cancel_alpha_refill", low=1., high=2.)
    # cancel_alpha = trial.suggest_float("cancel_alpha", low=2., high=8.)
    # cancel_alpha_all_refill = trial.suggest_float("cancel_alpha_all_refill", low=1., high=2.)
    # cancel_alpha_all = trial.suggest_float("cancel_alpha_all", low=2., high=8.)

    # invest_limit = trial.suggest_int("invest_limit", low=800, high=950)
    # cancel_limit = trial.suggest_int("cancel_limit", low=950, high=990)
    # invest_cost = trial.suggest_int("invest_cost", low=400, high=600)
    work_single_beta = trial.suggest_float("work_single_beta", low=1.1, high=3.)
    cancel_limit = 979
    invest_cost = 510
    invest_limit = 863
    # work_single_beta = 2.

    solver_path = "../target/release/ahc029"
    args = f"{overflow_alpha_refill} {overflow_alpha} {cancel_alpha_refill} {cancel_alpha} {invest_limit} {cancel_limit} {invest_cost} {overflow_alpha_all_refill} {overflow_alpha_all} {cancel_alpha_all_refill} {cancel_alpha_all} {work_single_beta}"
    solver_cmd = f"../tools/target/release/tester {solver_path} {args}"
    solver_version = f"optuna-{args}"
    runner = Runner(
        Input,
        Result,
        Param,
        solver_cmd=solver_cmd,
        solver_version=solver_version,
        database_csv="../log/database.csv",
        log_file="../log/a.log",
        verbose=0
    )
    runner.run(
        cases=cases,
        verbose=0
    )
    score_df = runner.evaluate_relative_score(
        solver_version=solver_version,
        benchmark_solver_version="tuned2",
    )
    return score_df.relative_score.median()

subprocess.run("cargo build --release --manifest-path=../Cargo.toml", shell=True)

study = optuna.create_study(
    direction = "maximize",
    storage=f"sqlite:///ahc029.db",
    study_name=f"study-work-single-beta-01",
    load_if_exists=True
)
print("start optimize")
study.optimize(objective, n_trials=30, show_progress_bar=True)
study.best_params

 --> src/solver.rs:3:5
  |
3 | use crate::simulator::*;
  |     ^^^^^^^^^^^^^^^^^^^
  |
  = note: `#[warn(unused_imports)]` on by default

 --> src/simulator.rs:7:4
  |
7 | fn generate_project(rng: &mut ChaCha20Rng) -> Project {
  |    ^^^^^^^^^^^^^^^^
  |
  = note: `#[warn(dead_code)]` on by default

  --> src/simulator.rs:17:4
   |
17 | fn generate_card(rng: &mut ChaCha20Rng, m: usize, x: &Vec<i64>) -> (Card, i64) {
   |    ^^^^^^^^^^^^^

  --> src/simulator.rs:59:12
   |
58 | impl MockInteractor {
   | ------------------- associated function in this implementation
59 |     pub fn new(
   |            ^^^

   --> src/simulator.rs:165:8
    |
165 | pub fn montecarlo(
    |        ^^^^^^^^^^

    Finished release [optimized] target(s) in 0.02s
[I 2023-12-26 18:32:54,534] A new study created in RDB with name: study-work-single-beta-01


start optimize


  0%|          | 0/30 [00:00<?, ?it/s]

7117.56s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
7117.57s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
7117.59s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
7117.61s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
7117.62s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
7117.63s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
7117.64s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
7117.65s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


[I 2023-12-26 18:33:15,253] Trial 0 finished with value: 1.0300310603687945 and parameters: {'work_single_beta': 2.0}. Best is trial 0 with value: 1.0300310603687945.
[I 2023-12-26 18:33:29,741] Trial 1 finished with value: 0.9108238807503 and parameters: {'work_single_beta': 1.58147796152504}. Best is trial 0 with value: 1.0300310603687945.
[I 2023-12-26 18:33:44,210] Trial 2 finished with value: 1.0289490499200271 and parameters: {'work_single_beta': 2.929862848031764}. Best is trial 0 with value: 1.0300310603687945.
[I 2023-12-26 18:33:58,571] Trial 3 finished with value: 0.9434852544074332 and parameters: {'work_single_beta': 1.6579686106992417}. Best is trial 0 with value: 1.0300310603687945.
[I 2023-12-26 18:34:12,940] Trial 4 finished with value: 0.9425772776554077 and parameters: {'work_single_beta': 1.8436043140701397}. Best is trial 0 with value: 1.0300310603687945.
[I 2023-12-26 18:34:27,269] Trial 5 finished with value: 0.9460552378106517 and parameters: {'work_single_beta'

{'work_single_beta': 2.898910615818517}

In [32]:
optuna.visualization.plot_slice(study)

In [31]:
optuna.visualization.plot_optimization_history(study)